In [1]:
# Importing necessary libraries
from random import seed, randint
from numpy import array, argmax
from math import ceil, log10
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM, TimeDistributed, RepeatVector


In [2]:
def random_sum_pairs(n_examples, n_numbers, largest): # Define a function to create random sum pairs
    X, y = [], []
    for i in range(n_examples): # Loop through the number of examples
        # Generate random numbers for the input pattern
        in_pattern = [randint(1, largest) for _ in range(n_numbers)]
        # Calculate the output pattern by summing the input pattern
        out_pattern = sum(in_pattern)
        X.append(in_pattern) # Append the input pattern to X
        y.append(out_pattern) # Append the output pattern to y
    return X, y


In [3]:
# Generate random sum pairs
X, y = random_sum_pairs(100, 4, 20) # Call the function to generate 100 examples with 4 numbers up to 20
print(X) # Print the input patterns
print(y) # Print the output patterns

In [4]:
# Calculate the maximum length required for input and output sequences
ceil(log10(20+1))

In [5]:
def pairs_to_string(X, y, n_numbers, largest): # Define a function to convert pairs to string
    max_length = n_numbers * ceil(log10(largest+1)) + n_numbers - 1 # Calculate the maximum length
    Xstr = []
    # Convert input patterns to string
    for p in X:
        strp = '+'.join([str(n) for n in p]) # Join the numbers with '+'
        strp = ''.join([' ' for _ in range(max_length - len(strp))]) + strp # Add padding
        Xstr.append(strp) # Append to Xstr
    max_length = ceil(log10(n_numbers * (largest+1))) # Calculate the maximum length for output
    ystr = []
    # Convert output patterns to string
    for p in y:
        strp = str(p)
        strp = ''.join([' ' for _ in range(max_length - len(strp))]) + strp # Add padding
        ystr.append(strp) # Append to ystr
    return Xstr, ystr


In [6]:
# Convert pairs to string
pairs_to_string(X, y, 4, 20)

In [7]:
# Extract X and y strings
Xstr, ystr = pairs_to_string(X, y, 4, 20)

In [8]:
def integer_encode(X, y, alphabets): # Define a function to encode strings into integers
    char_to_int = dict((c, i) for i, c in enumerate(alphabets)) # Create character to integer mapping
    Xenc = []
    # Encode X strings
    for p in X:
        integer_encoded = [char_to_int[char] for char in p] # Map characters to integers
        Xenc.append(integer_encoded) # Append to Xenc
    yenc = []
    # Encode y strings
    for p in y:
        integer_encoded = [char_to_int[char] for char in p] # Map characters to integers
        yenc.append(integer_encoded) # Append to yenc
    return Xenc, yenc


In [9]:
# Integer encode X and y strings
integer_encode(Xstr, ystr, ['1', '2', '3', '4', '5', '6', '7', '8', '9', '0', ' ', '+'])

In [10]:
def one_hot_encode(X, y, max_int): # Define a function to perform one-hot encoding
    Xenc = []
    # One-hot encode X
    for p in X:
        pattern = []
        for index in p:
            vector = [0 for _ in range(max_int)] # Create zero vector
            vector[index] = 1 # Set appropriate index to 1
            pattern.append(vector) # Append to pattern
        Xenc.append(pattern) # Append to Xenc
    yenc = []
    # One-hot encode y
    for p in y:
        pattern = []
        for index in p:
            vector = [0 for _ in range(max_int)] # Create zero vector
            vector[index] = 1 # Set appropriate index to 1
            pattern.append(vector) # Append to pattern
        yenc.append(pattern) # Append to yenc
    return Xenc, yenc


In [11]:
def generate_data(n_samples, n_numbers, largest, alphabets): # Define a function to generate data
    X, y = random_sum_pairs(n_samples, n_numbers, largest) # Generate random sum pairs
    X, y = pairs_to_string(X, y, n_numbers, largest) # Convert pairs to string
    X, y = integer_encode(X, y, alphabets) # Integer encode strings
    X, y = one_hot_encode(X, y, len(alphabets)) # One-hot encode strings
    X, y = array(X), array(y) # Convert to numpy arrays
    return X, y


In [12]:
# Define a function to decode integer sequences to strings
def integer_decode(seq, alphabets):
    int_to_char = dict((i, c) for i, c in enumerate(alphabets)) # Create integer to character mapping
    strings = []
    # Decode integer sequences
    for p in seq:
        string = int_to_char[argmax(p)] # Get corresponding character for the maximum value
        strings.append(string) # Append to strings
    return ''.join(strings) # Concatenate strings


In [13]:
# Set random seed and define parameters
seed(1)
n_samples = 1000
n_numbers = 3
largest = 20
alphabets = ['1', '2', '3', '4', '5', '6', '7', '8', '9', '0', ' ', '+']
n_chars = len(alphabets)
n_in_seq_length = n_numbers * ceil(log10(largest+1)) + n_numbers - 1
n_out_seq_length = ceil(log10(n_numbers * (largest+1)))


In [14]:
# Generate data
X, y = generate_data(n_samples, n_numbers, largest, alphabets)


In [15]:
# Define the model architecture
model = Sequential()
model.add(LSTM(128, input_shape=(n_in_seq_length, n_chars)))
model.add(RepeatVector(n_out_seq_length))
model.add(LSTM(64, return_sequences=True))
model.add(TimeDistributed(Dense(n_chars, activation='softmax')))


In [16]:
# Compile the model
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])


In [17]:
# Train the model
for i in range(200):
    X, y = generate_data(n_samples, n_numbers, largest, alphabets)
    print('Epoch: ', i)
    model.fit(X, y, epochs=5, batch_size=50)


In [18]:
# Generate data for evaluation
X, y = generate_data(n_samples, n_numbers, largest, alphabets)
# Predict the output
result = model.predict(X, batch_size=50)


In [19]:
# Decode the predicted and expected outputs
expected = [integer_decode(y1, alphabets) for y1 in y]
predicted = [integer_decode(y1, alphabets) for y1 in result]


In [20]:
# Calculate the absolute difference between expected and predicted outputs
import numpy as np
error_rate = np.abs(np.array(expected, dtype='int') - np.array(predicted, dtype='int')).sum() / len(expected)
error_rate # Print the error rate
